In [3]:
from stsci.tools import capable
capable.OF_GRAPHICS = False

from pyraf import iraf
from pyraf.iraf import gemini, gemtools, gmos, onedspec
import fileSelect as fs
import copy, os
from astropy.io import fits


In [4]:
pwd

u'/home/mmarcano/Documents/GeminiReduceTTU/Object14'

First specified the directory where all the data was downloaded and extracted. Then defined the sqlite3 database. This contains the information of all the files. This obsLog.sqlite3 file was created by the python script obslog.py in the raw data directory. The script will create a database with the fields and metadata

In [5]:
datadirall = '../raw/'
databasename = datadirall+'obsLog.sqlite3'
dbFile= datadirall+'obsLog.sqlite3'

 From the work_directory:
 Create the query dictionary of essential parameter=value pairs.

In [6]:
qd = {'Full':{'use_me':1,
       'Instrument':'GMOS-S','CcdBin':'2 2','RoI':'Full',
       'Disperser':'R400+_G5325','CentWave':646.0,'AperMask':'0.5arcsec',
       'Object':'Obj-14',
       'DateObs':'2018-09-15:2018-09-30'}
      }


 In the tutorial there are two regions but here only one and it is full. Just changed the regions list to only Full. It should work. Bias only depend on:
 instrument Roi and CcdBin according to fileSelect file
 
     use_me=1 AND ObsType='BIAS' AND ObsClass LIKE '%Cal'
    AND Instrument=:Instrument AND RoI=:RoI AND CcdBin=:CcdBin

In [7]:
gemtools.gemextn.unlearn()    # Disarm a bug in gbias
gmos.gbias.unlearn()
biasFlags = {
    'logfile':'biasLog.txt','rawpath':'../raw/','fl_vardq':'yes','verbose':'no'
}
regions = ['Full']
for r in regions:
    # The following SQL generates the list of full-frame files to process.
    SQL = fs.createQuery('bias', qd[r])
    biasFiles = fs.fileListQuery(dbFile, SQL, qd[r])

    # The str.join() funciton is needed to transform a python list into a
    # comma-separated string of file names that IRAF can understand.
    if len(biasFiles) > 1:
        gmos.gbias(','.join(str(x) for x in biasFiles), 'MCbias'+r,
                   **biasFlags)

# Flat Field

In [8]:
# Set the task parameters.
qd['Full'].update({'DateObs':'*'})
gmos.gireduce.unlearn()
gmos.gsflat.unlearn()
# The response fitting should be done interactively.
flatFlags = {
    'fl_over':'yes','fl_trim':'yes','fl_bias':'yes','fl_dark':'no',
    'fl_fixpix':'no','fl_oversize':'no','fl_vardq':'yes','fl_fulldq':'yes',
    'rawpath':'../raw/','fl_inter':'no','fl_detec':'yes',
    'function':'spline3','order':'13,11,28',
    'logfile':'gsflatLog.txt','verbose':'no'
    }
for r in regions:
    qr = qd[r]
    flatFiles = fs.fileListQuery(dbFile, fs.createQuery('gcalFlat', qr), qr)
    if len(flatFiles) > 0:
        gmos.gsflat (','.join(str(x) for x in flatFiles), 'MCflat'+r,
                bias='MCbias'+r, **flatFlags)


GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    

# Basic Reduction

In [9]:
# Set task parameters.
gmos.gsreduce.unlearn()
sciFlags = {
    'fl_over':'yes','fl_trim':'yes','fl_bias':'yes','fl_gscrrej':'no',
    'fl_dark':'no','fl_flat':'yes','fl_gmosaic':'yes','fl_fixpix':'no',
    'fl_gsappwave':'yes','fl_oversize':'no',
    'fl_vardq':'yes','fl_fulldq':'yes','rawpath':'../raw/',
    'fl_inter':'no','logfile':'gsreduceLog.txt','verbose':'no'
}
arcFlags = copy.deepcopy(sciFlags)
arcFlags.update({'fl_flat':'no','fl_vardq':'no','fl_fulldq':'no'})


# Arc exposures
for r in regions:
    qr = qd[r]
    arcFiles = fs.fileListQuery(dbFile, fs.createQuery('arc', qr), qr)
    if len(arcFiles) > 0:
        gmos.gsreduce (','.join(str(x) for x in arcFiles), bias='MCbias'+r,
                  **arcFlags)


# Science exposures
r = 'Full'
sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qd[r]), qd[r])
if len(sciFiles) > 0:
    gmos.gsreduce (','.join(str(x) for x in sciFiles), bias='MCbias'+r,
              flatim='MCflat'+r, **sciFlags)

GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 0.5arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged


# Wavelength Calibration (Check this later)

**Image rectification and wavelength linearization depend upon the wavelength calibration, using the arc lamp exposures taken immediately before each sequence of science and standard star exposures** (see Wavelength Calibration). In this case, the default medium-resolution line list will work well. The fit to the dispersion relation should be performed interactively, but for expediency we will use a previously determined functional fit. 


I had tpo manually put the name of the two arc Files. I am not sure how may to use. 


- [ ] On that day there is only one arc


Science ones are:

- S20181004S0022.fits 
- S20181004S0022.fits 

at 23:53 and 23:55

The closet arc is on 2018-10-04 called :S20181004S0209.fits

## But his depends on CentWave

for other one do:

S20181004S0210.fits

In [18]:
arcname = 'S20180922S0023'

In [19]:
# Set task parameters
gmos.gswavelength.unlearn()
waveFlags = {
    'coordlist':'gmos$data/CuAr_GMOS.dat','fwidth':6,'nsum':50,
    'function':'chebyshev','order':5,
    'fl_inter':'no','logfile':'gswaveLog.txt','verbose':'no'
    }
# Must select specific wavecals to match science exposures.
prefix = 'gs'
for arc in [arcname+'.fits']:
     gmos.gswavelength (prefix+arc, **waveFlags)

# Advanced Processing


In the example tutorial:

>The targets in this program were observed in 3 slit orientations, and a few exposures were obtained at each position. This provides an opportunity to combine the sequential exposures at each position to remove cosmic rays, rather than rejecting CRs on single frames using the gsreduce.fl_gscrrej+ flag or running the gemcrspec task. The combined exposures for each target are then wavelength calibrated, and sky subtracted. First set the processing parameters.

Here we have two exposures of the same thing. Supposed it is the same. 

In [20]:
# Set task parameters.
gemtools.gemcombine.unlearn()
sciCombFlags = {
    'combine':'average','reject':'ccdclip',
    'fl_vardq':'yes','fl_dqprop':'yes',
    'logfile':'gemcombineLog.txt.txt','verbose':'no'
}
stdCombFlags = copy.deepcopy(sciCombFlags)
stdCombFlags.update({'fl_vardq':'no','fl_dqprop':'no'})
gmos.gstransform.unlearn()
transFlags = {
    'fl_vardq':'yes','interptype':'linear','fl_flux':'yes',
    'logfile':'gstransLog.txt'
}
# The sky regions should be selected with care, using e.g. prows/pcols:
#   pcols ("tAM2306b.fits[SCI]", 1100, 2040, wy1=40, wy2=320)
gmos.gsskysub.unlearn()
skyFlags = {
    'fl_oversize':'no','fl_vardq':'yes','logfile':'gsskysubLog.txt'
}

# Science Targets

have to put the bad colum in *list one* to fix an also manually change the arc and sky postion to substract.

In [34]:
sciTargets = {
    'Obj-14':{'arc':'gs'+arcname,'sky':'100:700'},
}

for targ,p in sciTargets.iteritems():
    qs = qd['Full']
    qs['Object'] = targ
    # Fix up the target name for the output file
    #sciOut = targ.split('-')[0]+targ[-1]
    sciOut = targ.split('-')[0]+targ.split('-')[1]
    sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qs), qs)
    gemtools.gemcombine (','.join(prefix+str(x) for x in sciFiles),
                         sciOut, **sciCombFlags)

ERROR - GEMCOMBINE: Output file Obj14 already exists.


In [38]:
sciTargets = {
    'Obj-14':{'arc':'gs'+arcname,'sky':'1696:1740'},
}

for targ,p in sciTargets.iteritems():

    qs = qd['Full']
    qs['Object'] = targ
    # Fix up the target name for the output file
    sciOut = targ.split('-')[0]+targ.split('-')[1]
    sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qs), qs)
    
    
    #interpolate
    scienceobect = sciOut+'.fits'
    
    a = fits.open(sciOut+'.fits')
    axis1 = a[1].header['NAXIS1']
    axis2 = a[1].header['NAXIS2']


    #listone=[[1900 , 1970, 1, 2088],[2370,2395, 1, 2088],[300,320, 1, 2088]]
    listone=[[2822 , 2826, 1, 2088],[1172,1180,1,2088],[2370,2400,1,2088]]

    # x1 x2 y1 y2
    filemaskbad = 'maskbadcol.txt'
    filemaskiraf = 'maskbadcol.pl'
    with open(filemaskbad,'w') as f:
        for l in listone:
            x1,x2,y1,y2 = l
            f.write('{} {} {} {}\n'.format(x1,x2,y1,y2))
    iraf.text2mask(filemaskbad,filemaskiraf,axis1,axis2)

    iraf.copy(scienceobect,'tempstObj.fits')
    iraf.proto.fixpix('tempstObj.fits[SCI,1]',filemaskiraf,linterp='1,2,3,4',verbose='yes',pixel='no')
    iraf.copy('tempstObj.fits','e'+scienceobect)
    
    iraf.imarith('maskbadcol.pl', '+', 'e'+scienceobect+'[dq,1]', \
                 'tmpdq'+'e'+scienceobect)
    iraf.imcopy('tmpdq'+'e'+scienceobect+'[*,*]','e'+scienceobect+'[dq,1][*,*]' )
    os.remove('tempstObj.fits')
    os.remove('tmpdq'+'e'+scienceobect)


    
    gmos.gstransform ('e'+scienceobect, wavtraname=p['arc'], **transFlags)
    gmos.gsskysub ('t'+'e'+scienceobect, long_sample=p['sky'], **skyFlags)


FIXPIX: image tempstObj.fits[SCI,1] with mask maskbadcol.pl
tmpdqeObj14.fits[*,*] -> eObj14.fits[dq,1][*,*]
-------------------------------------------------------------------------------
GSTRANSFORM -- Fri Nov 30 13:56:37 CST 2018
 

inimages   = eObj14.fits
outimages  = 
outprefix  = t
fl_stran   = no
fl_wavtran = yes
wavtraname = gsS20180922S0023
database   = database
fl_vardq   = yes
interptype = linear
lambda1    = INDEF
lambda2    = INDEF
dx         = INDEF
nx         = INDEF
lambdalog  = no
ylog       = no
fl_flux    = yes
gratingdb  = gmos$data/GMOSgratings.dat
filterdb   = gmos$data/GMOSfilters.dat
key_dispaxis = DISPAXIS
dispaxis   = 1
sci_ext    = SCI
var_ext = VAR
dq_ext  = DQ

Transforming eObj14.fits[SCI,1]
MDF row: 1

NOAO/IRAF V2.16 mmarcano@tux Fri 13:56:38 30-Nov-2018
  Transform eObj14.fits[SCI,1] to tmpsci5790_2210.
  Conserve flux per pixel.
  User coordinate transformations:
    gsS20180922S0023_001
  Interpolation is linear.
  Using edge extension for out of boun

After sky substract fix column again

In [81]:
def fixcol(file,listcols):
    """Carefi; woth naxis. Should chedk with header is SCI"""
    a = fits.open(file)
    axis1 = a[2].header['NAXIS1']
    axis2 = a[2].header['NAXIS2']
    
    scienceobect = file


    #listone=[[1900 , 1970, 1, 2088],[2370,2395, 1, 2088],[300,320, 1, 2088]]
    listone=[[2370,2400,1,2088]]

    # x1 x2 y1 y2
    filemaskbad = 'maskbadcol.txt'
    filemaskiraf = 'maskbadcol.pl'
    with open(filemaskbad,'w') as f:
        for l in listone:
            x1,x2,y1,y2 = l
            f.write('{} {} {} {}\n'.format(x1,x2,y1,y2))
    iraf.text2mask(filemaskbad,filemaskiraf,axis1,axis2)

    iraf.copy(scienceobect,'tempstObj.fits')
    iraf.proto.fixpix('tempstObj.fits[SCI,1]',filemaskiraf,linterp='1,2,3,4',verbose='yes',pixel='no')
    iraf.copy('tempstObj.fits','e'+scienceobect)
    
    iraf.imarith('maskbadcol.pl', '+', 'e'+scienceobect+'[dq,1]', \
                 'tmpdq'+'e'+scienceobect)
    iraf.imcopy('tmpdq'+'e'+scienceobect+'[*,*]','e'+scienceobect+'[dq,1][*,*]' )
    os.remove('tempstObj.fits')
    os.remove('tmpdq'+'e'+scienceobect)

In [82]:
fixcol('steObj6.fits',[[2370,2400,1,2088]])

FIXPIX: image tempstObj.fits[SCI,1] with mask maskbadcol.pl
tmpdqesteObj6.fits[*,*] -> esteObj6.fits[dq,1][*,*]


## Try

In [71]:
onedspec.nsum=2
onedspec.sarith('esteObj6.fits[SCI]', 'copy', '', 'esteObj6.ms',
                  apertures='1037-1060x2')


pcols tObj5.fits[SCI] 1500 1700 wy1=40 

prows tAM2306b.fits[SCI] 520 720 wcs='world'



Sky http://www.gemini.edu/sciops/telescopes-and-sites/observing-condition-constraints/optical-sky-background